In [ ]:
import csv
from jobspy import scrape_jobs

# jobs = scrape_jobs(
#     site_name=["indeed", "linkedin", "zip_recruiter", "glassdoor"],
#     search_term="software engineer",
#     location="Dallas, TX",
#     results_wanted=20,
#     hours_old=72, # (only Linkedin/Indeed is hour specific, others round up to days old)
#     country_indeed='USA',  # only needed for indeed / glassdoor
#     # linkedin_fetch_description=True # get full description and direct job url for linkedin (slower)
# )
# print(f"Found {len(jobs)} jobs")
# print(jobs.head())
# jobs.to_csv("jobs.csv", quoting=csv.QUOTE_NONNUMERIC, escapechar="\\", index=False) # to_xlsx

In [ ]:
import pandas as pd


data = pd.read_csv("/mnt/c/Users/Green.PC/Documents/ASAMA/project/reflex-recommendation-job/notebook/jobs.csv")
data.head()

In [ ]:
data.isna().sum()

In [ ]:
# job_filter = jobs.loc[:, (jobs.isna().sum() / jobs.shape[0] < 0.25)]
# job_filter.head()
# job_filter.shape

In [ ]:
# job_filter.T

# Embeddings

In [ ]:
import openai

In [13]:
def get_embeddings(text):
    if isinstance(text, str):  # Check if text is a string
        response = openai.Embedding.create(
            model="text-embedding-ada-002",
            input=text.replace("\n", " ")
        )
        embedding = response['data'][0]['embedding']
        return embedding
    else:
        return None 

# ETL

In [ ]:
import json
import pandas as pd
import openai
import os
from loguru import logger

import psycopg2
from pgvector.psycopg2 import register_vector
import numpy as np
from psycopg2.extras import execute_values
import numpy as np
from dotenv import load_dotenv
load_dotenv()

In [14]:
def etl():
    jobs = scrape_jobs(
        site_name=["indeed", "linkedin", "zip_recruiter", "glassdoor"],
        search_term="software engineer",
        location="San Francisco",
        results_wanted=20,
        hours_old=72, # (only Linkedin/Indeed is hour specific, others round up to days old)
        country_indeed='USA',  # only needed for indeed / glassdoor
        # linkedin_fetch_description=True # get full description and direct job url for linkedin (slower)
    )

    logger.info(f"{jobs.shape}")
    

    # Load
    connection = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="mypass",
        host="localhost",
        port="5432"
    )
    
    cur = connection.cursor()
    #install pgvector
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
    connection.commit()
    
    
    register_vector(connection)
    
    # Create table to store embeddings and metadata
    table_create_command = """
    CREATE TABLE IF NOT EXISTS embeddings (
                id bigserial primary key, 
                site text,
                job_url text,
                title text,
                company text,
                location text,
                date_posted text,
                interval text,
                min_amount double precision,
                max_amount double precision,
                currency text,
                is_remote text,
                description text,
                company_url text,
                company_logo text,
                embedding vector(1536)
                );
                """
    cur.execute(table_create_command)
    cur.close()
    connection.commit()
    
    #Batch insert embeddings and metadata from dataframe into PostgreSQL database
    register_vector(connection)
    cur = connection.cursor()
    # Prepare the list of tuples to insert
    # Prepare the list of tuples to insert
    
    data_list = [
        (
            row['site'],
            row['job_url'],
            row['title'],
            row['company'],
            row['location'],
            row['date_posted'],
            row['interval'],
            row['min_amount'],
            row['max_amount'],
            row['currency'],
            row['is_remote'],
            row['description'],
            row['company_url'],
            row['company_logo'],
            np.array(get_embeddings(row['description']))  # Assuming 'embeddings' is a column with vector data
        )
        for index, row in jobs.iterrows()
    ]

    # Use execute_values to perform batch insertion
    execute_values(
        cur,
        """
        INSERT INTO embeddings (site, job_url, title, company, location, date_posted, interval, min_amount, max_amount, currency, is_remote, description, company_url, company_logo, embedding)
        VALUES %s
        """,
        data_list
    )
    # Commit after we insert all embeddings
    connection.commit()

In [15]:
etl()

2024-11-12 16:13:26,350 - INFO - JobSpy:Indeed - search page: 1 / 1
2024-11-12 16:13:26,353 - INFO - JobSpy:LinkedIn - search page: 1 / 2
2024-11-12 16:13:27,708 - INFO - JobSpy:ZipRecruiter - search page: 1 / 1
2024-11-12 16:13:28,058 - ERROR - JobSpy:ZipRecruiter - ZipRecruiter response status code 403 with response: {"request_id":"CFRAY:8e169b97adca8d53-IAD","error_code":"geoblocked-gdpr","status_code":403,"error_message":"This app is not available in the European Union, which enforces the General Data Protection Regulation (GDPR). Because of this regulation, ZipRecruiter cannot provide access at this time."}
2024-11-12 16:13:28,061 - INFO - JobSpy:ZipRecruiter - finished scraping
2024-11-12 16:13:28,641 - INFO - JobSpy:Glassdoor - search page: 1 / 1
2024-11-12 16:13:29,373 - INFO - JobSpy:Indeed - finished scraping
2024-11-12 16:13:31,443 - INFO - JobSpy:Glassdoor - finished scraping
2024-11-12 16:13:59,036 - ERROR - JobSpy:LinkedIn - LinkedIn: HTTPSConnectionPool(host='www.linkedi